<a href="https://colab.research.google.com/github/AmeerHamza0220/MachineLearningBasic/blob/main/nlp_getting_starting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
! kaggle competitions download -c nlp-getting-started

In [ ]:
!unzip nlp-getting-started.zip

In [ ]:
import pandas as pd
import numpy as np
import gensim
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
tqdm.pandas(desc="progress-bar")

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data=pd.read_csv("train.csv")
data

In [ ]:
def tokenize(tweet):
    try:
        tokens = tokenizer.tokenize(tweet)
        return tokens
    except:
        return 'NC'

In [ ]:
data['tokens']=data['text'].progress_map(tokenize)

In [ ]:
data

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(data['text'])


In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs') 
log_reg.fit(bow,data['target'])
log_reg.score(bow,data['target'])

In [ ]:
tfid=TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfid_x=tfid.fit_transform(data['text'])
log_reg.fit(tfid_x,data['target'])
log_reg.score(tfid_x,data['target'])

In [ ]:
w2v =Word2Vec(
            size=200, # desired no. of features/independent variables
)
w2v.build_vocab([i for i in tqdm(data['tokens'])])

In [ ]:
w2v.train(data['tokens'], total_examples= len(data['text']), epochs=4)

In [ ]:
w2v.most_similar('killed')

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

wordvec_arrays = np.zeros((len(data['tokens']), 200)) 
for i in range(len(data['tokens'])):
    wordvec_arrays[i,:] = word_vector(data['tokens'][i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

In [ ]:
log_reg.fit(wordvec_df,data['target'])
log_reg.score(wordvec_df,data['target'])

In [ ]:
bow.shape

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(100, activation='tanh', input_dim=200))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(wordvec_arrays, data['target'], epochs=9, batch_size=32, verbose=2)